# Policy iteration


As a part of our quest to better understand reinforcement learning, we experiment with basic dynamic programming algorithms. 

In a previous experiment, presented in [another notebook](./maze_value_iteration.ipynb) , we covered _Value iteration_, another key dynamic programming algorithm. We illustrated an example implementation of the algorithm on a simple _grid world_ or _maze_. 

In addition, we introduced the necessary terminology and background for dynamic programming, briefly discussing for example Markov Decision Processess, policies and value functions.

This notebook continues the quest by implementing _Policy Itaration_ algorithm and illustrates the algorithm using the same examples as before. As the basics were already covered in the notebook, they are not repeated here, although we lightly elaborate the concepts specific for our implementation.

As before, implementation of the common `Maze`building blocks are imported from another notebook, see [here](./maze_basis.ipynb).

In [ ]:
import math
import numpy as np

import ipynb.fs

from ipynb.fs.defs.maze_basis import (
    Maze, Movement, 
    plot_maze, plot_policy_actions, plot_state_rewards, plot_state_values,
)

## Brief recap of policy evaluation

As [discussed](./maze_value_iteration.ipynb#Policy-evaluation-and-update-rules), the task of computing state-value function for states under policy $\pi$ is called _policy evaluation_.

The _state-value function_ of a state $s$ under policy $\pi$, denoted $V^{\pi }$, is the expected discounted return when following the policy from state $s$ onwards:

\begin{equation*}
V^{\pi }\!\left ( s \right ) =
\mathbb{E}_{\pi}\!\left [
    U_{t}\vert S_{t}=s
\right ]
\end{equation*}

The _Bellman expectation equation_ for $V^{\pi }$, gives the relation of the value of a state and its successor states as: 

\begin{equation*}
V^{\pi }\!\left ( s \right ) =
\sum_{{s}'\in\mathcal{S}}
P\left( {s}' \vert s,\pi \left(s \right ) \right )
\left( r\left (s,\pi \left(s \right ),{s}'\right )  + \gamma 
V^{\pi } \left ( {s}' \right )  \right ) 
\end{equation*}

As noted, we have an equation for each state $V^{\pi}\!\left ( s \right )$ that refers to values of subsequent states $V^{\pi}\!\left ( {s}' \right )$. For all states $s \in \mathcal{S}$, gives a system of $\left |\mathcal{S}\right |$ simultaneous linear equations with $\left |\mathcal{S}\right |$ unknowns.

## Matrix implementation of policy evaluation

To solve the system using a matrix equation, we first define the following matrixes:


> $\mathbf{V}^{\pi}$ is a $\left |\mathcal{S}\right | \times 1$ vector of state values where $\mathit{v_{j}}=V^{\pi }\left ( s_{j} \right )$

> $\mathbf{R^{\pi}}$ is a $\left |\mathcal{S}\right | \times 1$ vector of rewards $\mathit{r_{j}}= r\left( s,\pi \left ( s \right ),s_{j}\right)$, when $S_{t}=s,A_{t}=\pi \left ( s \right ),S_{t+1}=s_{j}$.

> $\mathbf{P}^{\pi }$ is a $\left |\mathcal{S}\right | \times \left |\mathcal{S}\right |$ matrix of transition probabilities  where each element $\mathit{p_{jk}}$ represents $\mathbb{P}\!\left ( S_{t+1}=s_{k}\vert S_{t}=s_{j},A_{t}=\pi \left ( s_{j} \right ) \right )$

Using the above, we can represent the Bellman expectation equation in a matrix form as

\begin{equation*}
\mathbf{V}^{\pi}=\mathbf{P}^{\pi} \left (\mathbf{R}^{\pi}+\gamma \mathbf{V}^{\pi} \right )
\end{equation*}

\begin{equation*}
\mathbf{V}^{\pi}=\mathbf{P}^{\pi}\mathbf{R}^{\pi}+\gamma \mathbf{P}^{\pi}\mathbf{V}^{\pi}
\end{equation*}

Solving for $\mathbf{V}^{\pi}$ yields

\begin{equation*}
\mathbf{V}^{\pi}=\left (\mathbf{I}-\gamma \mathbf{P}^{\pi} \right )^{-1} \mathbf{P}^{\pi}\mathbf{R}^{\pi}
\end{equation*}

Note that the term $\mathbf{P}^{\pi}\mathbf{R}^{\pi}$ is a $\left |\mathcal{S}\right |$ dimensional vector that represents the expected immediate return received from state $s$ when following policy $\pi$

\begin{equation*}
\mathbb{E}^{\pi}\!\left ( R_{t+1}\vert S_{t}=s,A_{t}=\pi \left ( s \right ) \right ) =
\sum_{{s}'\in\mathcal{S}}
P\left( {s}' \vert s,\pi \left ( s \right ) \right )
r\left ( s,\pi \left ( s \right ),{s}' \right ) 
\end{equation*}

where we are taking advantage of the fact that in our simple maze problem, the rewards depend only on the state entered, and we can use a vector to represent the returns.

The selected approach sets us to invert a $\left |\mathcal{S}\right | \times \left |\mathcal{S}\right |$ matrix $\mathbf{I}-\gamma \mathbf{P}^{\pi} $, an operation with computational complexity of $O(\left |\mathcal{S}\right |^{3})$. An alternative iterative approach for policy evaluation we chose not to follow - using the Bellman expectation equation as an update rule, the _Bellman expectation backup_ - was brefly discussed [earlier](maze_value_iteration.ipynb#Policy-evaluation-and-update-rules).

## Policy iteration algorithm

_Policy iteration_ in an incremental algorithm to find the optimal policy $\pi^{\ast}$ for a fully defined MDP. The algorithm alternates between two steps, _policy evaluation_ and _policy improvement_. 

Policy evaluation step is performed as discussed above - we find the state values corresponding to current policy, the $V^{\pi }\!\left ( s \right )$ for all $s$. 

Policy improvement step then finds a new better policy ${\pi}'$ by determining the best action in each state with respect to the state-value function of the policy derived during the policy evaluation phase as:

\begin{equation*}
{\pi}'\!\left ( s \right ) = \underset{a}{\mathrm{argmax}}\;Q^{\pi}\! \left ( s, a \right ) 
\end{equation*}

The recursive relationship between $Q^{\pi }$ and $V^{\pi }$ discussed earlier enables us to determine ${\pi}'\!\left ( s \right )$ by using one-step look-ahead - referring to state-value function of subsequent states only. 

This phase of improvement is also described as '_acting greedily_' with respect to current policy and the resulting policy is a '_greedy policy_' with respect to the value function. Remember, that in the case of value iteration, a similar procedure was referred to as 'policy extraction'.

Policy iteration algorithm is guaranteed to converge starting from any arbitary policy: It can be shown, that each improvement step improves the value function, and also that if improvements stop, then the optimal value function $V^{\ast}\!\left ( s \right )$ satisfying the Bellman optimality equation, and the corresponding optimal policy $\pi^{\ast}\!\left ( s \right )$ have been found. 

As each iteration considers a changed and improved policy (unless the algorithm has converged), previously evaluated policies of lesser value are not revisited. Still, in theory, convergence could be slow as there are $\left |\mathcal{A}\right |^{\left |\mathcal{S}\right |}$ different policies to evaluate. In practice, convergence is often rapid.

Pseudocode for the algorithm stands as follows:

<span style="font-family: monospace;">

> **Policy iteration:**     
>
> initialize $k =0$, $V_{0}\left ( s \right ) \in \mathbb{R}$ and $\pi_{0}\!\left ( s \right ) \in \mathcal{A}$  for all $S$
> 
> loop:     
>> ***Policy evaluation:*** 
>>   
>> solve $
\mathbf{V}^{\pi}=\left (\mathbf{I}-\gamma \mathbf{P}^{\pi} \right )^{-1} \mathbf{P}^{\pi}\mathbf{R}^{\pi}
$ for $V_{k}\!\left ( s \right )$, using $\pi_{k}\!\left ( s \right )$
>>
>>
>> ***Policy improvement:*** 
>> 
>>    
>> for each $s \in \mathbb{S}$:
>>> improve policy, $
\pi_{k+1}\!\left ( s \right ) \leftarrow \underset{a}{\mathrm{argmax}}\; 
\sum_{{s}'\in\mathcal{S}}
P\left( {s}' \vert s,a \right )
\left( R\left (s,a,{s}'\right )  + \gamma 
V_{k} \left ( {s}' \right )  \right )
$
>>
>> $k = k + 1$  
>    
> until convergence, $\pi_{k+1}\!\left ( s \right ) = \pi_{k}\!\left ( s \right )$ for all $s \in \mathcal{S}$
> 
> return $\pi^{\ast} = \pi_{k+1}$

</span>

For the implementation, we need to maintain the current state values $V_{k}$ and the current policy $\pi_{k}$. Storing the previous policy for comparison as implied above is strictly not necessary if we take note whenever policy improvement changes policy for a state and terminate when policy is stable; i.e. there are no more changes to $\pi_{k+1}\!\left ( s \right )$.

Computational complexity is $O(\left |\mathcal{A}\right |\left |\mathcal{S}\right |^{2})$ per iteration for $\left |\mathcal{A}\right |$ actions and $\left |\mathcal{S}\right |$ states. However, solving the linear system adds a matrix inversion of complexity $O(\left |\mathcal{S}\right |^{3})$ for each iteration.

Substituting iterative policy evaluation would replace the evaluation step with the following:

<span style="font-family: monospace;">

> initialize $\theta > 0$, $j, k =0$, $V_{0}\left ( s \right ) \in \mathbb{R}$ and $\pi_{0}\!\left ( s \right ) \in \mathcal{A}$  for all $S$
>    
> ***Policy evaluation:*** 
>
>
> loop: 
>>    
>> for each $s \in \mathbb{S}$:
>>> update $
V_{j+1 }\!\left ( s \right ) \leftarrow
\sum_{{s}'\in\mathcal{S}}
P\left( {s}' \vert s,\pi_{k} \left(s \right ) \right )
\left( r\left (s,\pi_{k} \left(s \right ),{s}'\right )  + \gamma 
V_{j }\! \left ( {s}' \right )  \right ) 
$
>>
>> $j = j + 1$
>
> until convergence, $\left \| V_{j+1} - V_{j} \right \|_{\infty}<\theta$
</span>

## Policy iteration implementation

In the following, we illustrate an implementation of Policy Iteration algorithm. 

For clarity, rather than importing the notebook, we repeat shared code from Value Iteration algorithm implemetation here, but comment on the reused parts here.

First, `get_best_action` implementation is reused. The function that is used for state-value function calculation and for policy extraction in Value iteration is used for policy improvement step in the case of policy iteration:

\begin{equation*}
\pi_{k+1}\!\left ( s \right ) \leftarrow \underset{a}{\mathrm{argmax}}\; 
\sum_{{s}'\in\mathcal{S}}
P\left( {s}' \vert s,a \right )
\left( R\left (s,a,{s}'\right )  + \gamma 
V_{k} \left ( {s}' \right )  \right )
\end{equation*}

Note again that the implementation is lazy; For several equally good actions, the first one is returned.

In [ ]:
def get_best_action(maze, movement, state, state_values):
   
    best_action = None
    best_value = -math.inf

    for action in movement.actions:

        val = 0

        for move_direction, p_move in movement.get_direction_probs(action):

            s_prime = movement.move_from(state, move_direction)

            reward = maze.living_cost + maze.rewards[s_prime]
            s_prime_value = state_values[s_prime]

            val += p_move * (reward + maze.gamma * s_prime_value)

        if val > best_value:
            best_value = val
            best_action = action

    return best_value, best_action

`extract_policy` is again shared code. The function performs a sweep through states and finds the greedy action for each state based on state values determined during policy evaluation step

In [ ]:
def extract_policy(maze, movement, state_values):
    
    policy = {}
    
    for state in maze.get_iterator("states"):
        
        if maze.terminal[state]:
            continue
        
        best_value, best_action = get_best_action(maze, movement, state, state_values)

        policy[state] = best_action
    
    return policy

`calc_transfer_probabilities` utilizes the `Maze` structure to account for walls and `Movement` to account noisy movement, and generates the matrix of state transition probabilities $\mathbf{P}^{\pi }$ for current policy $\pi$

In [ ]:
def calc_transfer_probabilities(maze, movement, policy):
    
    def get_state_index(state):
        for z, s in enumerate(maze.get_iterator('states')):
            if s_prime == s:
                return z
    
    k = maze.state_count
    p = np.zeros((k,k))

    for s_index, state in enumerate(maze.get_iterator('states')):

        if maze.terminal[state]:
            continue

        policy_action = policy[state]

        for move_direction, p_move in movement.get_direction_probs(policy_action):
            
            s_prime = movement.move_from(state, move_direction)
            s_prime_index = get_state_index(s_prime)
            
            p[s_index, s_prime_index] += p_move

    return p

`policy_evaluation` implements the evaluation step of the algorithm for current policy in matrix form. Inversion of the matrix is performed using the Moore-Penrose pseudo-inverse as implemented in `numpy.linalg.pinv`.

In [ ]:
def policy_evaluation(maze, movement, policy):
    
    rewards = np.array( [ maze.get_as_list('rewards') ] ).T
    rewards += maze.living_cost
    
    k = maze.state_count
    
    p = calc_transfer_probabilities(maze, movement, policy)
    
    a = (np.eye(k) - maze.gamma * p)
    
    # print(f"matrix a shape: {a.shape}, rank: {np.linalg.matrix_rank(a)}, det: {np.linalg.det(a)}, cond: {np.linalg.cond(a)}")

    b = p @ rewards

    # path one: pseudoinverse
    a_inv = np.linalg.pinv(a)
    new_values = a_inv @ b

    # path two: linalg.solve
    # new_values = np.linalg.solve(a, b)
    
    # path three: least squares method
    #new_values, residuals, rank, sv = np.linalg.lstsq(a, b, rcond=None)
    
    new_values_dict = {}

    for i, state in enumerate(maze.get_iterator('states')):
        new_values_dict[state] = new_values[i,0]
    
    return new_values_dict

the 'acting-greedily' step of `policy_improvement` gently hides the fact that we are actually reusing policy extraction implementation from earlier

In [ ]:
def policy_improvement(maze, movement, state_values):
    
    # when doing value iteration, this function is called extract_policy
    
    return extract_policy(maze, movement, state_values)

we could retain `distance` as a concept, and change to `ord=0` for `np.linalg.norm`, yielding the desired number of changed policy elements between updates. This is not used currently, however.

In [ ]:
def distance(list1, list2):
    
#    return np.linalg.norm(np.subtract(list1, list2), ord=np.inf) # ord=np.inf for vectors is max(abs(x))
    
    return np.linalg.norm(np.subtract(list1, list2), ord=0) # ord=0 for vectors is sum(x != 0)

The entry point to main loop allows us to terminate early based on max number of iterations (`MAX_ITERS`). 

Again, we may choose to plot intermediate results after each iteration by giving a suitable plotter function for `plotter` keyword argument, such as `plot_maze` utility function defined in Maze basis implementation notebook. As the policy is updated and available after each iteration, we can readily visualize both intermediate state-values and intermediate policy if so desired.

In [ ]:
def policy_iteration(maze, movement, initial_policy, *, MAX_ITERS=10, plotter=None):
    
    policy = initial_policy.copy()
    
    for i in range(0, MAX_ITERS): 
       
        values = policy_evaluation(maze, movement, policy)
        updated_policy = policy_improvement(maze, movement, values)

        if plotter:
            plotter(maze, values, updated_policy)

        changed_count = sum([ policy[r] != updated_policy[r] for r in maze.get_iterator("states") if r in policy and r in updated_policy ])

        print(f"ROUND {i}, {changed_count}")

        if changed_count == 0:
            return values, policy

        policy = updated_policy
    
    return values, policy

## Example: The Canonical maze

Follow the flow used with Value Iteration algorithm, we first illustrate the algorithm using the canonical maze as an example and, as before, intialize the maze with single wall cell and two final states. We start with `gamma=1`, `living_cost = -0.04` and `noise=0.2`: 

In [ ]:
maze_config = {
    'size': (3, 4),
    'walls': [(1,1)],
    'terminal_states': [(0,3), (1,3)],
    'rewards': {
        (0,3): 1,
        (1,3): -1
    }
}

In [ ]:
maze = Maze(maze_config, gamma=1, living_cost = -0.04)
movement = Movement(maze, noise=0.2)

We use initial state-values of zero and extract the policy corresponding to initial values as initial policy. 

Note that the initial values are not needed by the algorithm, but are only used to create the initial policy. Thus we hope to have a similar starting point for the algorithm as we had with policy iteration earlier.

In [ ]:
initial_values = { state:0 for state in maze.get_iterator("states") }
initial_policy = extract_policy(maze, movement, initial_values)

plot_maze(maze, initial_values, initial_policy)

Now ready to run the algorithm, we hope to converge in 20 iterations and use `plot_maze` visualize state-values and corresponding greedy policy actions for each iteration step below.

In [ ]:
values, policy = policy_iteration(maze, movement, initial_policy,
                                  MAX_ITERS=20, plotter=plot_maze)

In [ ]:
display(values)
display({ r:Movement.action_names[policy[r]] for r in maze.get_iterator("states") if r in policy})

## Example 2: a slightly more complicated maze

Repeating our second example for policy iteration, we run through corresponding steps for a maze of medium complexity. We again use learning parameters `gamma=1`, `living_cost=-0.04` and `noise=0.2`.

In [ ]:
maze_config2 = {
    'size': (5, 7),
    'walls': [(1,1), (1,2), (1,3), (2,1), (3,1),
              (3,3), (3,4), (3,5), (2,5), (3,5) ],
    'terminal_states': [(2,3), (1,5)],
    'rewards': {
        (2,3): 1,
        (1,5): -1
    }
}
maze2 = Maze(maze_config2, gamma=1, living_cost = -0.04)
movement2 = Movement(maze2, noise=0.2)

plot_state_rewards(maze2, ax=None)

We initialize by finding an initial policy based on state value values of zero.

This time, we omit the `plotter` argument and thus skip plotting intermediate value functions or policies.

In [ ]:
initial_values2 = { state:0 for state in maze2.get_iterator("states") }
initial_policy2 = extract_policy(maze2, movement2, initial_values2)

In [ ]:
values2, policy2 = policy_iteration(maze2, movement2, initial_policy2, MAX_ITERS=20)

In [ ]:
plot_state_values(maze2, values2)

In [ ]:
plot_policy_actions(maze2, policy2)

## Final example: a complex maze

As our third and final example, we tackle the same ludicrously complex maze as before, but this time using policy iteration.

In [ ]:
maze_config3 = {
    'size': (8, 7),
    'walls': [ (1,1), (1,2), (1,4), 
              (2,1), (2,4), 
              (4,3),(4,5), (4,6),
              (5,2),
              (6,3), (6,4), (6,5),
              (7,1) ],
    'terminal_states': [ (1,5), (2,2), (2,5), (4,1), (4,2), (5,1), (5,3) ],
    'rewards': {
        (1,5): -1,
        (2,2): -1,
        (2,5): -1,
        (4,1): -1,
        (4,2): -1,
        (5,3):  1,
        (5,1): -1,
    }
}
maze3 = Maze(maze_config3, gamma=0.9, living_cost = -0.01)
movement3 = Movement(maze3, noise=0.2)

initial_values3 = { state:0 for state in maze3.get_iterator("states") }
initial_policy3 = extract_policy(maze3, movement3, initial_values3)

In [ ]:
plot_state_rewards(maze3, ax=None)

In [ ]:
values3, policy3 = policy_iteration(maze3, movement3, initial_policy3, MAX_ITERS=50)

In [ ]:
plot_state_values(maze3, values3)

In [ ]:
plot_policy_actions(maze3, policy3)

## Timing the execution

To get in idea of relative computational cost, we time the execution of the algorithm for the final example.

In [ ]:
%%capture c
%%timeit

values3, policy3 = policy_iteration(maze3, movement3, initial_policy3, MAX_ITERS=50)

In [ ]:
import re
regex = r'.*$'
timeitresults = re.findall(regex, c.stdout)
print(timeitresults[0])

A [result for Value iteration](./maze_value_iteration.ipynb#Timing-it) was 

68.4 ms +- 1.85 ms per loop (mean +- std. dev. of 7 runs, 10 loops each)

for ` C_LIMIT = 0.0001`, resulting in 39 iterations of the algorithm. This should be compared with 5 iterations needed for policy iteration above.

Exact comparison of the two algorithms is difficult, however, as value iteration terminates based on value of $\theta$ and the policy probably has converged before the state-values reach the target level of maximum change between iterations - we could use a significant share of computation time to fine-tune low decimals of value-function estimates. 

Nevertheless, the computational cost of policy iteration compares favorably despite the burden of matrix inversion during policy evaluation step. 